# Additional preprocessing

In [2]:
import pandas as pd
import us
import numpy as np
from tqdm import tqdm

In [17]:
# Data from Census API calls
cen = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data/census_final.csv')

/Users/alexandrabruno/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
list(cen['YEAR'].unique())

[2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [3]:
cen

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,YEAR
0,0,PWGTP,AGEP,FER,SEX,RACBLK,RACWHT,RACSOR,HISP,SCHL,ST,2004
1,1,83,90,0,2,0,1,0,1,13,21,2004
2,2,220,37,2,2,0,0,0,2,9,6,2004
3,3,222,14,0,1,0,0,0,2,5,6,2004
4,4,221,12,0,1,0,0,0,2,4,6,2004
...,...,...,...,...,...,...,...,...,...,...,...,...
47568301,3239549,68,43,2,2,1,0,0,1,21,24,2019
47568302,3239550,82,13,0,1,1,0,0,1,9,24,2019
47568303,3239551,82,10,0,1,1,0,0,1,7,24,2019
47568304,3239552,82,5,0,1,1,0,0,1,3,24,2019


In [4]:
# Deleting Unnamed: 0 column
del cen['Unnamed: 0']

# Creating a list of our column names 
new_header = [
    'PWGTP',
    'AGEP',
    'FER',
    'SEX',
    'RACBLK',
    'RACWHT',
    'RACSOR',
    'HISP',
    'SCHL',
    'ST',
    'YEAR'
]

# Setting our column names equal to our list 
cen.columns = new_header

In [5]:
# Get indexes where name column has value PWGTP
indexNames = cen[cen['PWGTP'] == 'PWGTP'].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

In [6]:
# Change columns into the data types we need so we can use them for creating our features
cen.FER = cen.FER.astype(int)
cen.HISP = cen.HISP.astype(int)
cen.RACWHT = cen.RACWHT.astype(int)
cen.RACBLK = cen.RACBLK.astype(int)
cen.AGEP = cen.AGEP.astype(int)
cen.ST = cen.ST.astype(str)
cen.SCHL = cen.SCHL.astype(int)

In [7]:
# Get indexes where name column has value 0
indexNames = cen[cen['FER'] == 0].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value 8
indexNames = cen[cen['FER'] == 8].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value > 46
indexNames = cen[cen['AGEP'] >= 46].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value < 14 
indexNames = cen[cen['AGEP'] <= 14].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

In [8]:
# Creating a list of the fip codes
fip_codes = ['1','2','4','5','6','8','9','10','11','12','13','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']

In [9]:
# Creating a list of the states
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois',
          'Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland',
         'Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana',
         'Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island',
          'South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington',
          'West Virginia','Wisconsin','Wyoming']

In [10]:
# Zipping our list into a dicitionary so we can replace the old values with the new ones 
changes = dict(zip(fip_codes,states))

In [11]:
# Replacing the values with a lambda function 
cen['STATE'] = cen['ST'].apply(lambda x: changes[x])

In [12]:
cen['EDUC'] = pd.cut(cen.SCHL, [1,16,18,21,42], labels=['less hs','hs','som col','col or more'],right=False)

In [13]:
# Making our education variables dummies
cen = pd.get_dummies(cen, columns=['EDUC'])

In [14]:
cen = cen.rename(columns={'EDUC_col or more': 'COLPLUS', 'EDUC_hs':'HS', 'EDUC_less hs': 'LESSHS','EDUC_som col':'SOMCOL'})

In [15]:
cen['HISPAN'] = np.where(cen['HISP']>1, 1, 0)

In [16]:
states = list(cen['STATE'].unique())
years = list(cen['YEAR'].unique())

In [31]:
lst = []
for state in tqdm(states):
    for year in years:
        try:
            perc_blk = cen.loc[(cen['YEAR']==year) & (cen['RACBLK']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
            cen.loc[(cen['YEAR']==year) & (cen['RACBLK']== 1) & (cen['STATE']== state) , 'PRC_BLK'] = perc_blk
        except:
            lst.append([state,year]) 
            continue

100%|██████████| 51/51 [14:56<00:00, 17.59s/it]


In [33]:
lst = []
for state in tqdm(states):
    for year in years:
        try:
            perc_wht = cen.loc[(cen['YEAR']==year) & (cen['RACWHT']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
            cen.loc[(cen['YEAR']==year) & (cen['RACWHT']== 1) & (cen['STATE']== state), 'PRC_WHT'] = perc_wht
        except:
            lst.append([state,year]) 
            continue

100%|██████████| 51/51 [16:53<00:00, 19.88s/it]


In [35]:
lst = []
for state in tqdm(states):
    for year in years:
        try:
            perc_hisp = cen.loc[(cen['YEAR']==year) & (cen['HISPAN']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
            cen.loc[(cen['YEAR']==year) & (cen['HISPAN']== 1) & (cen['STATE']== state), 'PRC_HISP'] = perc_hisp
        except:
            lst.append([state,year]) 
            continue

100%|██████████| 51/51 [14:03<00:00, 16.54s/it]


In [36]:
lst = []
for state in tqdm(states):
    for year in years:
        try:
            perc_other = cen.loc[(cen['YEAR']==year) & (cen['RACSOR']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
            cen.loc[(cen['YEAR']==year) & (cen['RACSOR']== 1) & (cen['STATE']== state), 'PRC_OTHER'] = perc_other
        except:
            lst.append([state,year]) 
            continue 

100%|██████████| 51/51 [25:17<00:00, 29.76s/it]


In [37]:
lst = []
for state in tqdm(states):
    for year in years:
            try:
                lesshs = cen.loc[(cen['YEAR']==year) & (cen['LESSHS']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
                cen.loc[(cen['YEAR']==year) & (cen['LESSHS']== 1) & (cen['STATE']== state), 'LESS_HS'] = lesshs
            except:
                lst.append([state,year]) 
                continue

100%|██████████| 51/51 [14:02<00:00, 16.52s/it]


In [39]:
lst = []
for state in tqdm(states):
    for year in years:
            try:
                hs = cen.loc[(cen['YEAR']==year) & (cen['HS']==1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
                cen.loc[(cen['YEAR']==year) & (cen['HS']==1) & (cen['STATE']== state), 'HS'] = hs
            except:
                lst.append([state,year]) 
                continue

100%|██████████| 51/51 [13:42<00:00, 16.12s/it]


In [40]:
lst = []
for state in tqdm(states):
    for year in years:
            try:
                somecol = cen.loc[(cen['YEAR']==year) & (cen['SOMCOL']==1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
                cen.loc[(cen['YEAR']==year) & (cen['SOMCOL']==1) & (cen['STATE']== state), 'SOM_COL'] = somecol
            except:
                lst.append([state,year]) 
                continue

100%|██████████| 51/51 [12:11<00:00, 14.34s/it]


In [41]:
lst = []
for state in tqdm(states):
    for year in years:
            try:
                colplus = cen.loc[(cen['YEAR']==year) & (cen['COLPLUS']== 1) & (cen['STATE']== state)]['FER'].value_counts(normalize = True)[1]
                cen.loc[(cen['YEAR']==year) & (cen['COLPLUS']== 1) & (cen['STATE']== state), 'COL_PLUS'] = colplus
            except:
                lst.append([state,year]) 
                continue 

100%|██████████| 51/51 [12:05<00:00, 14.23s/it]


In [42]:
cen

,PWGTP,AGEP,FER,SEX,RACBLK,RACWHT,RACSOR,HISP,SCHL,ST,...,SOMCOL,COLPLUS,HISPAN,PRC_BLK,PRC_WHT,PRC_HISP,PRC_OTHER,LESS_HS,SOM_COL,COL_PLUS
2,220,37,2,2,0,0,0,2,9,6,...,0,0,1,NaN,NaN,0.075705,NaN,0.062723,NaN,NaN
7,234,19,2,2,0,1,0,1,9,37,...,0,0,0,NaN,0.062472,NaN,NaN,0.063560,NaN,NaN
17,239,17,2,2,1,0,0,1,6,39,...,0,0,0,0.058370,NaN,NaN,NaN,0.057158,NaN,NaN
22,774,19,2,2,0,1,0,1,9,17,...,0,0,0,NaN,0.058595,NaN,NaN,0.059651,NaN,NaN
31,18,22,2,2,0,1,0,1,13,56,...,0,0,0,NaN,0.055619,NaN,NaN,0.060377,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47568277,70,42,2,2,0,1,0,1,1,17,...,0,0,0,NaN,0.054056,NaN,NaN,0.027510,NaN,NaN
47568278,61,31,2,2,0,1,0,1,23,11,...,0,1,0,NaN,0.029215,NaN,NaN,NaN,NaN,0.039106
47568281,56,25,2,2,0,1,0,2,19,6,...,1,0,1,NaN,0.053039,0.057099,NaN,NaN,0.053225,NaN
47568287,145,23,1,2,1,0,0,1,16,36,...,0,0,0,0.051309,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cen = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data_test/one.csv')

In [4]:
states = list(cen['STATE'].unique())
years = list(cen['YEAR'].unique())
x = len(states)*len(years)
cols = ['YEAR', 'STATE', 'PRC_BLK', 'PRC_WHT', 'PRC_HISP', 'PRC_OTHER']
df = pd.DataFrame(columns = cols, index = list(range(x)))


for state in tqdm(states):
    for year in years:
        row = cen[cen['STATE'] == state][cen['YEAR']==year].bfill().drop_duplicates().dropna()
        df = pd.concat([df,row]).dropna()
    
df

  0%|          | 0/51 [00:00<?, ?it/s]<ipython-input-4-5f2f3845d98e>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = cen[cen['STATE'] == state][cen['YEAR']==year].bfill().drop_duplicates().dropna()
100%|██████████| 51/51 [1:18:37<00:00, 92.50s/it] 


,YEAR,STATE,PRC_BLK,PRC_WHT,PRC_HISP,PRC_OTHER,Unnamed: 0,PWGTP,AGEP,FER,...,SCHL,ST,LESSHS,HS,SOMCOL,COLPLUS,HISPAN,LESS_HS,SOM_COL,COL_PLUS
2015322,2008,California,0.066364,0.066055,0.082417,0.083607,10037266.0,84.0,18.0,2.0,...,16.0,6.0,0.0,0.074797,0.0,0.0,1.0,0.061313,0.065620,0.077315
2015348,2008,California,0.066364,0.066055,0.082417,0.083607,10037384.0,64.0,44.0,2.0,...,19.0,6.0,0.0,0.000000,1.0,0.0,0.0,0.061313,0.065620,0.077315
2015349,2008,California,0.066364,0.066055,0.082417,0.083607,10037385.0,64.0,15.0,2.0,...,12.0,6.0,1.0,0.000000,0.0,0.0,1.0,0.061313,0.065620,0.077315
2015350,2008,California,0.066364,0.066055,0.082417,0.083607,10037387.0,103.0,44.0,2.0,...,19.0,6.0,0.0,0.000000,1.0,0.0,0.0,0.061313,0.065620,0.077315
2015351,2008,California,0.066364,0.066055,0.082417,0.083607,10037388.0,80.0,33.0,2.0,...,19.0,6.0,0.0,0.000000,1.0,0.0,0.0,0.061313,0.065620,0.077315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9045296,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337,47555124.0,29.0,20.0,2.0,...,18.0,40.0,0.0,0.000000,1.0,0.0,0.0,0.038174,0.067907,0.076455
9045363,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337,47555507.0,137.0,28.0,2.0,...,17.0,40.0,0.0,0.082168,0.0,0.0,0.0,0.038174,0.067907,0.076455
9045712,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337,47557570.0,92.0,42.0,2.0,...,19.0,40.0,0.0,0.000000,1.0,0.0,0.0,0.038174,0.067907,0.076455
9045844,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337,47558326.0,100.0,44.0,2.0,...,12.0,40.0,1.0,0.000000,0.0,0.0,1.0,0.038174,0.067907,0.076455


In [5]:
df.to_csv('/Users/alexandrabruno/Documents/flatiron/data_test/two.csv')

In [6]:
df.columns

Index(['YEAR', 'STATE', 'PRC_BLK', 'PRC_WHT', 'PRC_HISP', 'PRC_OTHER',
       'Unnamed: 0', 'PWGTP', 'AGEP', 'FER', 'SEX', 'RACBLK', 'RACWHT',
       'RACSOR', 'HISP', 'SCHL', 'ST', 'LESSHS', 'HS', 'SOMCOL', 'COLPLUS',
       'HISPAN', 'LESS_HS', 'SOM_COL', 'COL_PLUS'],
      dtype='object')

In [9]:
df.drop(['Unnamed: 0', 'PWGTP', 'AGEP', 'FER', 'SEX', 'RACBLK', 'RACWHT',
       'RACSOR', 'HISP', 'SCHL', 'ST', 'LESSHS', 'HS', 'SOMCOL', 'COLPLUS',
       'HISPAN', 'LESS_HS', 'SOM_COL', 'COL_PLUS'],axis= 1,inplace=True)

In [10]:
df

,YEAR,STATE,PRC_BLK,PRC_WHT,PRC_HISP,PRC_OTHER
2015322,2008,California,0.066364,0.066055,0.082417,0.083607
2015348,2008,California,0.066364,0.066055,0.082417,0.083607
2015349,2008,California,0.066364,0.066055,0.082417,0.083607
2015350,2008,California,0.066364,0.066055,0.082417,0.083607
2015351,2008,California,0.066364,0.066055,0.082417,0.083607
...,...,...,...,...,...,...
9045296,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337
9045363,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337
9045712,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337
9045844,2019,Oklahoma,0.082329,0.067695,0.057851,0.047337


In [11]:
df.drop_duplicates(inplace = True)

In [12]:
df

,YEAR,STATE,PRC_BLK,PRC_WHT,PRC_HISP,PRC_OTHER
2015322,2008,California,0.066364,0.066055,0.082417,0.083607
2600230,2009,California,0.067522,0.063556,0.079220,0.080778
3186480,2010,California,0.049433,0.060709,0.075456,0.079569
3774588,2011,California,0.055638,0.061056,0.071855,0.077166
4362283,2012,California,0.064666,0.058635,0.068064,0.071173
...,...,...,...,...,...,...
6108539,2015,Oklahoma,0.064220,0.058540,0.061377,0.057778
6693243,2016,Oklahoma,0.057377,0.067640,0.102985,0.086294
7276255,2017,Oklahoma,0.070342,0.072062,0.091549,0.099010
7867964,2018,Oklahoma,0.081395,0.063928,0.072983,0.056075


In [13]:
edu = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/maternal_mortality/edu_vars.csv')

In [15]:
_df = pd.merge(edu, df,  how='left', left_on=['STATE','YEAR'],right_on=['STATE','YEAR'])

In [16]:
_df

,Unnamed: 0,YEAR,STATE,LESS_HS,HS,SOM_COL,COL_PLUS,PRC_BLK,PRC_WHT,PRC_HISP,PRC_OTHER
0,0,2008,California,0.061313,0.074797,0.065620,0.077315,0.066364,0.066055,0.082417,0.083607
1,69886,2009,California,0.058627,0.074591,0.060827,0.073674,0.067522,0.063556,0.079220,0.080778
2,142071,2010,California,0.056496,0.072493,0.057649,0.067503,0.049433,0.060709,0.075456,0.079569
3,214623,2011,California,0.056581,0.065545,0.057849,0.072450,0.055638,0.061056,0.071855,0.077166
4,288201,2012,California,0.052276,0.067993,0.056351,0.069165,0.064666,0.058635,0.068064,0.071173
...,...,...,...,...,...,...,...,...,...,...,...
533,6762181,2015,Oklahoma,0.042539,0.067130,0.056548,0.066192,0.064220,0.058540,0.061377,0.057778
534,6768922,2016,Oklahoma,0.042215,0.083429,0.061734,0.084278,0.057377,0.067640,0.102985,0.086294
535,6775769,2017,Oklahoma,0.036090,0.085763,0.076065,0.083274,0.070342,0.072062,0.091549,0.099010
536,6782622,2018,Oklahoma,0.046887,0.061735,0.071308,0.081229,0.081395,0.063928,0.072983,0.056075


In [22]:
cdc = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data/cdc.csv')

In [23]:
cdc

,Unnamed: 0,YEAR,STATE,AGE,RTE_CHL,RTE_GON
0,0,2018,Alabama,35 to 39,115.1,79.0
1,1,2018,Alabama,45 to 49,286.0,178.4
2,2,2018,Alabama,15 to 19,653.5,325.5
3,3,2018,Alabama,20 to 24,1705.9,657.8
4,4,2018,Alabama,40 to 44,4513.2,1285.3
...,...,...,...,...,...,...
5809,5809,2000,Wyoming,45 to 49,43.6,0.0
5810,5810,2000,Wyoming,15 to 19,151.2,34.4
5811,5811,2000,Wyoming,20 to 24,404.7,41.9
5812,5812,2000,Wyoming,40 to 44,1615.2,43.7
